## Integración de base de datos

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida28/doctorado/ssev-analytics/terepaima
> Directorio actual:  /media/javier/Compartida28/doctorado/ssev-analytics


In [2]:
import pandas as pd

from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime

In [3]:
import warnings
warnings.filterwarnings('ignore')

Parque

In [4]:
park = 'terepaima'

DIR = f'./{park}/'

Data NARX precipitación

In [5]:
pd_narx_precipitacion = pd.read_pickle(f'{DIR}data/narx_precipitacion.pkl')
pd_narx_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media
0,terepaima,1995-01-01,1995,1,1,9.96,-69.38,training,0.230639,0.214150,1001.651794
1,terepaima,1995-02-01,1995,2,1,9.96,-69.38,training,0.216488,0.200263,1001.651794
2,terepaima,1995-03-01,1995,3,1,9.96,-69.38,training,2.480125,2.011823,1001.651794
3,terepaima,1995-04-01,1995,4,1,9.96,-69.38,training,2.167091,2.523585,1001.651794
4,terepaima,1995-05-01,1995,5,1,9.96,-69.38,training,2.611307,2.906471,1001.651794


Data ANN precipitación

In [6]:
pd_ann_precipitacion = pd.read_pickle(f'{DIR}data/ann_precipitacion_terepaima.pkl')
pd_ann_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann,ndvi_media
0,terepaima,1995-01-01,1995,1,1,9.96,-69.38,training,0.230639,0.214150,1001.651794,0.214150,0.378702,NaN
1,terepaima,1995-02-01,1995,2,1,9.96,-69.38,training,0.216488,0.200263,1001.651794,0.200263,0.463370,NaN
2,terepaima,1995-03-01,1995,3,1,9.96,-69.38,training,2.480125,2.011823,1001.651794,2.011823,1.193321,NaN
3,terepaima,1995-04-01,1995,4,1,9.96,-69.38,training,2.167091,2.523585,1001.651794,2.523585,2.667052,NaN
4,terepaima,1995-05-01,1995,5,1,9.96,-69.38,training,2.611307,2.906471,1001.651794,2.906471,3.381478,NaN


Data NARX NDVI

In [7]:
pd_narx_ndvi = pd.read_pickle(f'{DIR}data/narx_ndvi.pkl')
pd_narx_ndvi.periodo = pd.to_datetime(pd_narx_ndvi.periodo)
pd_narx_ndvi.head()

,id_point,periodo,type,ndvi_prediction,ndvi_media
0,2,2013-01-01,training,0.720235,0.746280
1,2,2013-02-01,training,0.703267,0.689997
2,2,2013-03-01,training,0.612171,0.503604
3,2,2013-04-01,training,0.575065,0.509868
4,2,2013-05-01,training,0.519919,0.402435


Integración de base de datos

In [8]:
pd_analisis = pd.merge(pd_ann_precipitacion[['park',
                                            'periodo',
                                            'year',
                                            'month',
                                            'id_point',
                                            'latitud',
                                            'longitud',
                                            'type',
                                            'precipitacion_mm',
                                            'precipitacion_narx',
                                            'prediction_ann',
                                            'elevacion_media']],
                        pd_narx_ndvi[['id_point', 'periodo','ndvi_prediction', 'ndvi_media']],
                        on=['id_point','periodo'],
                        how='left')

pd_analisis['time'] = pd_analisis['periodo'].apply(lambda x: datetime.toordinal(x))
pd_analisis['time_actualizacion'] = int(datetime.today().toordinal())
pd_analisis = pd_analisis[['park',
                        'year',
                        'month',
                        'id_point',
                        'latitud',
                        'longitud',
                        'type',
                        'precipitacion_mm',
                        'precipitacion_narx',
                        'prediction_ann',
                        'elevacion_media',
                        'ndvi_prediction',
                        'ndvi_media',
                        'time',
                        'time_actualizacion']]
pd_analisis.head()

,park,year,month,id_point,latitud,longitud,type,precipitacion_mm,precipitacion_narx,prediction_ann,elevacion_media,ndvi_prediction,ndvi_media,time,time_actualizacion
0,terepaima,1995,1,1,9.96,-69.38,training,0.230639,0.214150,0.378702,1001.651794,NaN,NaN,728294,738506
1,terepaima,1995,2,1,9.96,-69.38,training,0.216488,0.200263,0.463370,1001.651794,NaN,NaN,728325,738506
2,terepaima,1995,3,1,9.96,-69.38,training,2.480125,2.011823,1.193321,1001.651794,NaN,NaN,728353,738506
3,terepaima,1995,4,1,9.96,-69.38,training,2.167091,2.523585,2.667052,1001.651794,NaN,NaN,728384,738506
4,terepaima,1995,5,1,9.96,-69.38,training,2.611307,2.906471,3.381478,1001.651794,NaN,NaN,728414,738506


In [9]:
import json

json_data = '{ "data":' + pd_analisis.to_json(orient="records") +\
                ',"park" : "terepaima"' + "}"

with open(f'{DIR}data/json_data.json', 'w') as outfile:
    outfile.write(json_data)

In [10]:
# Mongo coleccion
documentos = json.loads( pd_analisis.to_json(orient="records") )

documentos[-1]

{'park': 'terepaima',
 'year': 2023,
 'month': 5,
 'id_point': 9,
 'latitud': 9.76,
 'longitud': -69.18,
 'type': 'prediction',
 'precipitacion_mm': None,
 'precipitacion_narx': 6.2704321086,
 'prediction_ann': 5.0195428577,
 'elevacion_media': 319.0556030273,
 'ndvi_prediction': 0.5969036276,
 'ndvi_media': None,
 'time': 738641,
 'time_actualizacion': 738506}

Conexión Mongo DB

In [11]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological',
 'meteorological',
 'estimateSSTNino34',
 'polygons',
 'SSTNino34']

In [12]:
# insertando coleccion
coleccion = db['estimateMeteorological']

In [13]:
# Insertandoss documentos
for doc in documentos:
  coleccion.update_one({"time":doc.get('time'),
                        "id_point":doc.get('id_point'),
                        "park":"terepaima"}, {"$set":doc}, upsert=True)